# Step 3: LLM Root Cause Analysis
Use Llama 3.2 (via Ollama) to analyze negative reviews and identify complaint categories

In [1]:
import requests
import pandas as pd
import json
from time import sleep
import os

print("Libraries loaded!")
print("\nMake sure Ollama is running with Llama 3.2:")
print("  Terminal command: ollama run llama3.2")

Libraries loaded!

Make sure Ollama is running with Llama 3.2:
  Terminal command: ollama run llama3.2


In [2]:
def analyze_with_ollama(review_text):
    """
    Analyze restaurant review using local Ollama LLM (Llama 3.2)
    Returns structured JSON with complaint categories and severity
    """
    
    prompt = f"""Analyze this restaurant review and identify complaint categories.
Return ONLY a valid JSON object with no other text.

Review: "{review_text}"

JSON format (use true/false for categories):
{{
  "food_quality": false,
  "service_speed": false,
  "staff_behavior": false,
  "cleanliness": false,
  "portion_size": false,
  "pricing": false,
  "order_accuracy": false,
  "severity": "low|medium|high",
  "primary_issue": "brief description"
}}

Set category to true if mentioned in the review. Severity: low=minor, medium=moderate, high=severe."""

    try:
        response = requests.post(
            'http://localhost:11434/api/generate',
            json={
                "model": "llama3.2",
                "prompt": prompt,
                "stream": False
            },
            timeout=30
        )
        
        if response.status_code != 200:
            return None
        
        result_text = response.json()['response']
        
        # Extract JSON from response
        json_start = result_text.find('{')
        json_end = result_text.rfind('}') + 1
        
        if json_start >= 0 and json_end > json_start:
            json_str = result_text[json_start:json_end]
            return json.loads(json_str)
        else:
            return None
            
    except Exception as e:
        print(f"Error: {e}")
        return None

In [3]:
# Test Ollama connection
print("Testing Ollama connection...\n")

test_review = "The food was cold and the waiter was very rude. We waited 45 minutes for our order."
test_result = analyze_with_ollama(test_review)

if test_result:
    print("✓ Ollama is working!")
    print(f"\nTest result:")
    print(json.dumps(test_result, indent=2))
else:
    print("❌ Ollama connection failed!")
    print("\nPlease check:")
    print("  1. Is Ollama running? (Terminal: ollama run llama3.2)")
    print("  2. Is port 11434 available?")
    print("  3. Is llama3.2 model installed? (ollama pull llama3.2)")

Testing Ollama connection...

✓ Ollama is working!

Test result:
{
  "food_quality": true,
  "service_speed": true,
  "staff_behavior": true,
  "cleanliness": false,
  "portion_size": false,
  "pricing": false,
  "order_accuracy": false,
  "severity": "low",
  "primary_issue": "The food was cold and the waiter was very rude. We waited 45 minutes for our order."
}


In [4]:

# Load feature-engineered data from optimized pipeline

df = pd.read_csv("data/processed/reviews_features_optimized.csv")
df['date'] = pd.to_datetime(df['date'])


print(f"Loaded data: {len(df):,} reviews")
print(f"\nNegative reviews (≤3 stars): {(df['stars_review'] <= 3).sum():,}")

Loaded data: 89,878 reviews

Negative reviews (≤3 stars): 27,933


In [5]:
# Sample negative reviews for analysis
# Note: Using smaller sample due to LLM processing time
sample_size = 20

negative_reviews = df[df['stars_review'] <= 3].sample(
    n=min(sample_size, len(df[df['stars_review'] <= 3])),
    random_state=42
)

print(f"Selected {len(negative_reviews)} negative reviews for LLM analysis")
print(f"\nRating distribution in sample:")
print(negative_reviews['stars_review'].value_counts().sort_index())

Selected 20 negative reviews for LLM analysis

Rating distribution in sample:
stars_review
1.0    10
2.0     1
3.0     9
Name: count, dtype: int64


In [6]:
# Analyze reviews with LLM
print("\nAnalyzing reviews with Llama 3.2...")
print("This may take 1-2 minutes (about 1 second per review)\n")

results = []
errors = 0

for idx, row in negative_reviews.iterrows():
    print(f"Analyzing review {len(results)+1}/{len(negative_reviews)}...", end='\r')
    
    analysis = analyze_with_ollama(row['text'])
    
    if analysis:
        # Add metadata
        analysis['business_id'] = row['business_id']
        analysis['business_name'] = row.get('name', 'Unknown')
        analysis['rating'] = row['stars_review']
        analysis['date'] = str(row['date'])
        analysis['review_text'] = row['text'][:200] + '...' if len(row['text']) > 200 else row['text']
        results.append(analysis)
    else:
        errors += 1
    
    # Rate limiting to avoid overloading local LLM
    sleep(0.5)

print(f"\n\n✓ Analysis complete!")
print(f"  Successful: {len(results)}/{len(negative_reviews)}")
print(f"  Errors: {errors}")


Analyzing reviews with Llama 3.2...
This may take 1-2 minutes (about 1 second per review)

Analyzing review 20/20...

✓ Analysis complete!
  Successful: 20/20
  Errors: 0


In [7]:
# Convert results to DataFrame
if len(results) > 0:
    complaint_df = pd.DataFrame(results)
    
    print(f"\nAnalyzed {len(complaint_df)} reviews")
    print(f"\nDataFrame columns: {complaint_df.columns.tolist()}")
else:
    print("❌ No results to analyze. Please check Ollama connection.")


Analyzed 20 reviews

DataFrame columns: ['food_quality', 'service_speed', 'staff_behavior', 'cleanliness', 'portion_size', 'pricing', 'order_accuracy', 'severity', 'primary_issue', 'business_id', 'business_name', 'rating', 'date', 'review_text']


In [ ]:
# Analyze complaint categories
if len(results) > 0:
    
    category_cols = [
        'food_quality', 'service_speed', 'staff_behavior',
        'cleanliness', 'portion_size', 'pricing', 'order_accuracy'
    ]
    
    # Filter to only existing columns
    available_cols = [col for col in category_cols if col in complaint_df.columns]
    
    if available_cols:
        category_flags = complaint_df[available_cols].applymap(
            lambda v: 1 if v is True else 0 
        )
        category_counts = category_flags.sum().sort_values(ascending=False)

        
        print("=" * 60)
        print("COMPLAINT CATEGORY DISTRIBUTION")
        print("=" * 60)
        for category, count in category_counts.items():
            percentage = (count / len(complaint_df)) * 100
            print(f"{category:20s}: {int(count):2d} reviews ({percentage:5.1f}%)")
    
    # Severity distribution
    if 'severity' in complaint_df.columns:
        print("\n" + "=" * 60)
        print("SEVERITY DISTRIBUTION")
        print("=" * 60)
        severity_counts = complaint_df['severity'].value_counts()
        for severity, count in severity_counts.items():
            percentage = (count / len(complaint_df)) * 100
            print(f"{severity:20s}: {int(count):2d} reviews ({percentage:5.1f}%)")
    
    # Top primary issues
    if 'primary_issue' in complaint_df.columns:
        print("\n" + "=" * 60)
        print("TOP 5 PRIMARY ISSUES")
        print("=" * 60)
        top_issues = complaint_df['primary_issue'].value_counts().head(5)
        for issue, count in top_issues.items():
            if issue and str(issue).strip():  # Skip empty issues
                print(f"  • {issue} ({int(count)} reviews)")

COMPLAINT CATEGORY DISTRIBUTION
food_quality        : 14 reviews ( 70.0%)
staff_behavior      :  7 reviews ( 35.0%)
portion_size        :  6 reviews ( 30.0%)
pricing             :  6 reviews ( 30.0%)
order_accuracy      :  3 reviews ( 15.0%)
cleanliness         :  2 reviews ( 10.0%)
service_speed       :  1 reviews (  5.0%)

SEVERITY DISTRIBUTION
low                 : 10 reviews ( 50.0%)
high                :  9 reviews ( 45.0%)
medium              :  1 reviews (  5.0%)

TOP 5 PRIMARY ISSUES
  • skimped on portion size of grilled salmon (1 reviews)
  • Rude staff who hung up the phone and claimed they had no reservations when asked about a reservation. (1 reviews)
  • Terrible food (1 reviews)
  • gluten free options (1 reviews)


/var/folders/j1/ln88kc6d3qn6v_mv1kg4krsc0000gn/T/ipykernel_9004/839512333.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  category_flags = complaint_df[available_cols].applymap(


In [10]:
# Save results
if len(results) > 0:
    output_file = 'results/llm_complaint_analysis.csv'
    complaint_df.to_csv(output_file, index=False)
    
    print("\n" + "=" * 60)
    print("RESULTS SAVED")
    print("=" * 60)
    print(f"✓ File: {output_file}")
    print(f"✓ Reviews analyzed: {len(complaint_df)}")
    print(f"✓ Columns: {complaint_df.shape[1]}")
    print(f"\nSample output:")
    print(complaint_df[['business_name', 'rating', 'severity', 'primary_issue']].head(3).to_string(index=False))
else:
    print("\n❌ No results to save")


RESULTS SAVED
✓ File: results/llm_complaint_analysis.csv
✓ Reviews analyzed: 20
✓ Columns: 14

Sample output:
            business_name  rating severity                                                                                  primary_issue
McGillin's Olde Ale House     3.0      low                                                                                               
           El Camino Real     1.0      low Staff were inattentive and failed to check in with customers, leading to a long wait for food.
                      Pod     3.0      low Dining off the main menu is too expensive and doesn't offer enough of a portion in my opinion.


In [12]:
# Summary statistics for report
if len(results) > 0:
    print("\n" + "=" * 60)
    print("SUMMARY FOR FINAL REPORT")
    print("=" * 60)
    print(f"\nLLM Implementation:")
    print(f"  - Model: Llama 3.2-3B (open-source)")
    print(f"  - Deployment: Local inference via Ollama")
    print(f"  - Sample analyzed: {len(complaint_df)} reviews")
    print(f"  - Success rate: {len(complaint_df)/len(negative_reviews)*100:.1f}%")
    print(f"  - Average processing time: ~1 second per review")
    print(f"  - Cost: $0 (local deployment)")
    
    if available_cols:
        print(f"\nTop Complaint Categories:")
        for i, (category, count) in enumerate(category_counts.head(3).items(), 1):
            percentage = (count / len(complaint_df)) * 100
            print(f"  {i}. {category}: {percentage:.0f}% ({int(count)} reviews)")
    
    print("\nKey Achievement:")
    print(" Zero-cost LLM deployment for detailed complaint categorization")
    print(" Enables targeted operational improvements")
    print(" Automated root cause identification")


SUMMARY FOR FINAL REPORT

LLM Implementation:
  - Model: Llama 3.2-3B (open-source)
  - Deployment: Local inference via Ollama
  - Sample analyzed: 20 reviews
  - Success rate: 100.0%
  - Average processing time: ~1 second per review
  - Cost: $0 (local deployment)

Top Complaint Categories:
  1. food_quality: 70% (14 reviews)
  2. staff_behavior: 35% (7 reviews)
  3. portion_size: 30% (6 reviews)

Key Achievement:
 Zero-cost LLM deployment for detailed complaint categorization
 Enables targeted operational improvements
 Automated root cause identification
